# 流形学习

流形学习是一种非线性降维方法。其算法基于的思想是：许多数据集维度过高的现象完全是人为导致得

高维数据集通常难以可视化。虽然,可以通过绘制两维或三维的数据来显示高维数据的固有结构，但与之等效的高维图不太直观。为了促进高维数据集结构的可视化，必须以某种方式降低维度。

通过对数据的随机投影来实现降维是最简单的方法。虽然这样做能实现数据结构一定程度的可视化，但这种随机选择方式仍有许多有待改进之处。在随机投影过程中，数据中更有趣的结构很可能会丢失。

为了解决这一问题，一些监督和无监督的线性降维框架被设计出来，如主成分分析（PCA），独立成分分析以及线性判别分析等。这些算法明确规定了如何来选择数据的“有趣的”线性投影。它们虽然高效，但是经常错失数据中重要的非线性结构。

流形学习可以被认为是将线性框架（如 PCA ）推广到对数据中非线性结构敏感的一次尝试。虽然存在监督变量，但是典型的流形学习问题是无监督的：它从数据本身学习数据的高维结构，而不使用预定的分类。

In [ ]:
from sklearn.datasets import make_s_curve
from sklearn.decomposition import PCA, KernelPCA

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

%matplotlib inline

PCA 的一个弱点是它无法检测到非线性特征。 已经开发了一组称为流形学习的算法，来解决这个缺陷。流形学习中使用的规范数据集是 S 曲线：

In [ ]:
X, y = make_s_curve(n_samples=1000)

ax = plt.axes(projection='3d')

ax.scatter3D(X[:, 0], X[:, 1], X[:, 2], c=y)
ax.view_init(10, -60)

这是一个嵌入三维的二维数据集，但它以某种方式嵌入，PCA 无法发现底层数据方向：

In [ ]:
X_pca = PCA(n_components=2).fit_transform(X)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y)

## Isomap
---

流形学习的最早方法之一是 Isomap 算法，等距映射（Isometric Mapping）的缩写。Isomap 可以被视为多维缩放（Multi-dimensional Scaling：MDS）或核主成分分析（Kernel PCA）的扩展。Isomap 寻求一个较低维度的嵌入( 译注：嵌入(embedding)，在此处，可以理解为高维数据到低维数据的一种映射转换，数据间的固有结构不变化 )，它保持了所有点之间的原有的测地距离( 译注:测地距离（geodesic distance）是指在图中连接某两个顶点的最短距离(shortest path) )。Isomap 可以通过 Isomap 对象执行。

Isomap 算法包括三个阶段:

1. 最近邻搜索. Isomap 使用 sklearn.neighbors.BallTree 进行有效的近邻搜索。 对于 D 维中 N 个点的 k 个最近邻，代价约为 $O[D \log(k) N \log(N)]$

2. 最短路径图搜索. 该类算法中已知最有效的算法是 Dijkstra 算法或 Floyd-Warshall 算法，其复杂度分别是约 $O[N^2(k + \log(N))]$ 和 $O[N^3]$ 。 用户可通过使用 isomap 的 path_method 关键字来选择该算法。 如果未指定，则代码自行尝试为输入数据选择最佳算法。

3. 部分特征值分解. 对应于 $N \times N$ isomap核中 d 个最大特征值的特征向量，进行嵌入编码。 对于密集求解器，代价约为 $O[d N^2]$ 。 通常可以使用 ARPACK 求解器来减少代价。 用户可通过使用 isomap 的 path_method 关键字指定特征求解器。 如果未指定，则代码自行尝试为输入数据选择最佳算法。

Isomap 的整体复杂度是 $O[D \log(k) N \log(N)] + O[N^2(k + \log(N))] + O[d N^2]$.

- $N$ : 训练数据点的个数
- $D$ : 输入维度
- $k$ : 最近邻的个数
- $d$ : 输出维度

In [ ]:
from sklearn.manifold import Isomap

X_iso = Isomap(n_neighbors=15, n_components=2).fit_transform(X)
plt.scatter(X_iso[:, 0], X_iso[:, 1], c=y)

**数字数据上的流形学习**
---
我们可以使用线性技术（例如 PCA）可视化数据集. 但是，使用更强大的非线性技术可以提供更好的可视化效果

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()

In [ ]:
pca = PCA(n_components=2)  # 64维降到2维
X_pca = pca.fit_transform(digits.data)

In [ ]:
# 10 个数字10种颜色, 不同类别是否明显可分 
def plot_pca_scatter(X_pca):
    plt.figure(figsize=(12, 10))
    colors = ['black', 'blue', 'purple', 'yellow', 'green',
              'red', 'lime', 'cyan', 'orange', 'gray']
    for i in range(10):
        plt.scatter(X_pca[digits.target==i, 0], X_pca[digits.target==i, 1],
                    color=colors[i], edgecolors='k')
    plt.legend(digits.target_names)
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Compoent')

In [ ]:
plot_pca_scatter(X_pca)

t-SNE（ TSNE ）将数据点的相似性转换为概率。原始空间中的相似性表示为高斯联合概率，嵌入空间中的相似性表示为 “学生” 的 t 分布。这允许 t-SNE 对局部结构特别敏感，并且有超过现有技术的一些其它优点:

- 在一个单一映射上按多种比例显示结构
- 显示位于多个、不同的流形或聚类中的数据
- 减轻在中心聚集的趋势

使用 t - SNE 的缺点大致如下:

- t-SNE 的计算成本很高，在百万样本数据集上可能需要几个小时，而PCA将在几秒或几分钟内完成同样工作。
- Barnes-Hut t-SNE 方法仅限于二维或三维嵌入。
- 该算法是随机的，不同种子的多次重新开始可以产生不同的嵌入。然而，以最小的误差选择嵌入是完全合理的。
- 未明确保留全局结构。用PCA初始化点(使用 init=’pca’ )，可以减轻此问题。

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(random_state=42)
digits_tsne = tsne.fit_transform(digits.data)
plt.figure(figsize=(10, 10))
plt.xlim(digits_tsne[:, 0].min(), digits_tsne[:, 0].max() + 1)
plt.ylim(digits_tsne[:, 1].min(), digits_tsne[:, 1].max() + 1)

colors = ['black', 'blue', 'purple', 'yellow', 'green',
          'red', 'lime', 'cyan', 'orange', 'gray']
for i in range(len(digits.data)):
    # 实际上将数字绘制为文本而不是使用散点图
    plt.text(digits_tsne[i, 0], digits_tsne[i, 1], str(digits.target[i]),
             color = colors[digits.target[i]],
             fontdict={'weight': 'bold', 'size': 9})

t-SNE 比其他流形学习算法运行时间更长，但结果非常惊人